In [ ]:
#pip install pyreadr

In [2]:
import pandas as pd
import pyreadr
from functools import reduce

# Data Cleaning

## 0. Delineation File

In [4]:
delineation_df = pd.read_excel('./data/delineation_file.xls')
delineation_df = delineation_df.iloc[1:1918]
delineation_df = delineation_df.rename(columns=delineation_df.iloc[0])
delineation_df = delineation_df.drop(1)
delineation_df['FIPS State Code'] = delineation_df['FIPS State Code'].astype(int)
delineation_df['CBSA Code'] = delineation_df['CBSA Code'].astype(int)
delineation_df = delineation_df[['CBSA Code', 'State Name', 'FIPS State Code', 'FIPS County Code']]
delineation_df

,CBSA Code,State Name,FIPS State Code,FIPS County Code
2,10100,South Dakota,46,013
3,10100,South Dakota,46,045
4,10140,Washington,53,027
5,10180,Texas,48,059
6,10180,Texas,48,253
...,...,...,...,...
1913,49700,California,6,101
1914,49700,California,6,115
1915,49740,Arizona,4,027
1916,49780,Ohio,39,119


In [5]:
code = pyreadr.read_r('./data/code.Rds')

In [6]:
code_df = code[None]
code_df['cityID'] = code_df['cityID'].astype(int)
code_df['countyID'] = code_df['countyID'].astype(int)
code_df = code_df.drop(columns=['NAME', 'ORI'])
code_df

,cityID,countyID,cbsa
0,100124,1067,20020
1,100460,1073,13820
2,100484,1133,NaN
3,1005690,10005,42580
4,1005820,10005,42580
...,...,...,...
22721,999999,9015,25540
22722,999999,9015,35980
22723,999999,9015,48740
22724,999999,9015,14860


In [7]:
code_merged = pd.merge(delineation_df, code_df, left_on='CBSA Code', right_on='cbsa', how='right')
code_merged.drop_duplicates()

,CBSA Code,State Name,FIPS State Code,FIPS County Code,cityID,countyID,cbsa
0,20020,Alabama,1.0,061,100124,1067,20020
1,20020,Alabama,1.0,067,100124,1067,20020
2,20020,Alabama,1.0,069,100124,1067,20020
3,13820,Alabama,1.0,007,100460,1073,13820
4,13820,Alabama,1.0,009,100460,1073,13820
...,...,...,...,...,...,...,...
73276,25540,Connecticut,9.0,013,999999,9015,25540
73277,35980,Connecticut,9.0,011,999999,9015,35980
73278,NaN,NaN,NaN,NaN,999999,9015,48740
73279,14860,Connecticut,9.0,001,999999,9015,14860


In [100]:
set(code_merged['State Name'].to_list())

{'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming',
 nan}

## 1. Crime Data

In [182]:
crime_df = pd.read_csv('./data/crime.csv')

In [334]:
crime = crime_df[['STATE', 'ORI', 'cityID', 'YEAR', 'MONTH', 'OFFENSE', 'OCCUR', 'POP']]
#crime_df = crime_df[crime_df['POP'] >= 3000]

for index, row in crime.iterrows():
    if crime['STATE'][index] == 'south' and crime['ORI'][index][0:2] == 'SC':
        crime['STATE'][index] = 'south carolina'
    if crime['STATE'][index] == 'south' and crime['ORI'][index][0:2] == 'SD':
        crime['STATE'][index] = 'south dakota'
    if crime['STATE'][index] == 'north' and crime['ORI'][index][0:2] == 'NC':
        crime['STATE'][index] = 'north carolina'
    if crime['STATE'][index] == 'north' and crime['ORI'][index][0:2] == 'ND':
        crime['STATE'][index] = 'north dakota'
    if crime['STATE'][index] == 'west':
        crime['STATE'][index] = 'west virginia'
    if crime['STATE'][index] == 'district':
        crime['STATE'][index] = 'district of columbia'
    if crime['STATE'][index] == 'new' and crime['ORI'][index][0:2] == 'NH':
        crime['STATE'][index] = 'new hampshire'
    if crime['STATE'][index] == 'new' and crime['ORI'][index][0:2] == 'NJ':
        crime['STATE'][index] = 'new jersey'
    if crime['STATE'][index] == 'new' and crime['ORI'][index][0:2] == 'NM':
        crime['STATE'][index] = 'new mexico'
    if crime['STATE'][index] == 'new' and crime['ORI'][index][0:2] == 'NY':
        crime['STATE'][index] = 'new york'
    if crime['STATE'][index] == 'rhode':
        crime['STATE'][index] = 'rhode island'
        

delineation_df_small = delineation_df[['FIPS State Code', 'State Name']]
delineation_df_small['State Name'] = delineation_df_small['State Name'].str.lower()
crime = pd.merge(crime, delineation_df_small, left_on='STATE',right_on='State Name', how='left')

lost_state = crime[crime['FIPS State Code'].isnull()]
lost_state_list = set(lost_state['STATE'].to_list())
print(lost_state_list)

crime = crime.drop(columns=['STATE', 'ORI', 'cityID', 'POP', 'OFFENSE', 'State Name'])
crime

/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/3501359597.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime['STATE'][index] = 'new jersey'
/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/3501359597.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime['STATE'][index] = 'new mexico'
/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/3501359597.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

{'guam'}


,YEAR,MONTH,OCCUR,FIPS State Code
0,2015,5,2,1.0
1,2015,5,2,1.0
2,2015,5,2,1.0
3,2015,5,2,1.0
4,2015,5,2,1.0
...,...,...,...,...
6844051,2014,7,2,39.0
6844052,2014,7,2,39.0
6844053,2014,7,2,39.0
6844054,2014,7,2,39.0


In [341]:
crime_agg = crime.groupby(['MONTH','YEAR','FIPS State Code']).sum().reset_index()
crime_agg = crime_agg.rename(columns={'MONTH': 'month','OCCUR': 'crime_counts'})
crime_agg['month'] = crime_agg['month'].astype(int)
crime_agg['YEAR'] = crime_agg['YEAR'].astype(int)
crime_agg = crime_agg.dropna(subset=['FIPS State Code'])
crime_agg

,month,YEAR,FIPS State Code,crime_counts
0,1,2012,2.0,250
1,1,2012,4.0,3108
2,1,2012,5.0,1681
3,1,2012,6.0,52110
4,1,2012,8.0,7395
...,...,...,...,...
2301,12,2015,51.0,28569
2302,12,2015,53.0,4928
2303,12,2015,54.0,1922
2304,12,2015,55.0,25400


## 2. Alcohol Data for 2012-2020

In [133]:
alcohol_17 = pd.read_csv('./data/alcohol/alcohol_17.csv')
alcohol_18 = pd.read_csv('./data/alcohol/alcohol_18.csv')
alcohol_19 = pd.read_csv('./data/alcohol/alcohol_19.csv')
alcohol_20 = pd.read_csv('./data/alcohol/alcohol_20.csv')
alcohol_17['YEAR'] = 2017
alcohol_18['YEAR'] = 2018
alcohol_19['YEAR'] = 2019
alcohol_20['YEAR'] = 2020
delineation_df_al = delineation_df[['CBSA Code', 'State Name', 'FIPS State Code']]
alcohol_17_20 = pd.concat([alcohol_17, alcohol_18, alcohol_19, alcohol_20])

new_df = alcohol_17_20['Geographic Area Name'].str.split(', ', expand=True)
alcohol_17_20['State_name'] = new_df[1]
alcohol_17_20.drop(columns =["Geographic Area Name"], inplace = True)
alcohol_17_20['alcohol'] = 0
alcohol_17_20 = alcohol_17_20.groupby(['YEAR', 'State_name']).count().reset_index()
alcohol_17_20['alcohol_per_month'] = round(alcohol_17_20['alcohol']/12, 2)
alcohol_17_20 = pd.merge(alcohol_17_20, delineation_df_al, left_on='State_name', right_on='State Name', how='left')
alcohol_17_20 = alcohol_17_20.dropna(subset=['FIPS State Code'])
alcohol_17_20 = alcohol_17_20.drop(columns=['alcohol', 'State_name', 'State Name', 'CBSA Code'])


alcohol_12_16 = pd.read_csv('./data/alcohol/alcohol_2012_2016.csv')
alcohol_12_16['alcohol_per_month'] = round(alcohol_12_16['alcohol']/12, 2)
alcohol_12_16 = alcohol_12_16.fillna(0)
alcohol_12_16 = pd.merge(alcohol_12_16, delineation_df_al, left_on='cbsa', right_on='CBSA Code', how='left')
alcohol_12_16 = alcohol_12_16.dropna(subset=['FIPS State Code'])
alcohol_12_16 = alcohol_12_16.drop(columns=['alcohol', 'State Name', 'cbsa', 'CBSA Code'])

alcohol_df = pd.concat([alcohol_12_16, alcohol_17_20])
alcohol_df = alcohol_df.groupby(['YEAR','FIPS State Code']).sum()
alcohol_df

alcohol_per_month
YEAR FIPS State Code                   
2012 1.0                          86.40
     2.0                           9.25
     4.0                          26.85
     5.0                          82.83
     6.0                         653.65
...                                 ...
2020 53.0                        105.00
     54.0                         44.02
     55.0                        200.00
     56.0                         23.87
     72.0                         81.00

[463 rows x 1 columns]

## 3. Twitter Data

In [323]:
twitter_df = pd.read_csv('./data/twitter_example.csv', encoding = "ISO-8859-1")
# only select useful columns
twitter_df = twitter_df[['id_str', 'UScounty', 'UScity', 'USstate', 'day', 'month', 'YEAR', 'user_id_str', 'lang',
                         'retweet_count', 'statuses_count', 'followers_count', 'favourites_count', 'friends_count']]
# drop missing value
twitter_df = twitter_df.dropna(subset = ['UScity', 'USstate', 'UScounty'], how='all')
# add state code
twitter_df = twitter_df.fillna(0)
twitter_df['UScity'] = twitter_df['UScity'].astype(int)
twitter_df['USstate'] = twitter_df['USstate'].astype(int)
twitter_df['UScounty'] = twitter_df['UScounty'].astype(int)
## deal with city
twitter_df = pd.merge(twitter_df, code_df, left_on='UScity', right_on='cityID', how='left')
twitter_df = twitter_df.drop_duplicates(subset=['id_str'])
## deal with county
twitter_df = pd.merge(twitter_df, code_df, left_on='UScounty', right_on='countyID', how='left')
twitter_df = twitter_df.drop_duplicates(subset=['id_str'])
## deal with state
delineation_df_small = delineation_df[['CBSA Code', 'FIPS State Code']]
twitter_df = pd.merge(twitter_df, delineation_df_small, left_on='cbsa_x', right_on='CBSA Code', how='left')
twitter_df = twitter_df.drop_duplicates(subset=['id_str'])
twitter_df = pd.merge(twitter_df, delineation_df_small, left_on='cbsa_y', right_on='CBSA Code', how='left')
twitter_df = twitter_df.drop_duplicates(subset=['id_str'])
twitter_df = twitter_df.drop(columns=['cityID_x', 'countyID_x', 'cbsa_x', 'cityID_y', 'countyID_y', 
                                      'cbsa_y', 'CBSA Code_x', 'CBSA Code_y'])
twitter_df = twitter_df.fillna(0)

for index, row in twitter_df.iterrows():
    if twitter_df['USstate'][index]==0 and twitter_df['FIPS State Code_x'][index]!=0:
        code = twitter_df['FIPS State Code_x'][index]
        twitter_df['USstate'][index] = code
    elif twitter_df['USstate'][index]==0 and twitter_df['FIPS State Code_y'][index]!=0:
        code = twitter_df['FIPS State Code_y'][index]
        twitter_df['USstate'][index] = code
twitter_df = twitter_df.drop(columns=['lang','FIPS State Code_x', 'FIPS State Code_y', 'UScounty', 
                                      'UScity', 'day', 'user_id_str', 'id_str'])
twitter_df = twitter_df[twitter_df['USstate']!=0]
twitter_df #4598

/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/3613544781.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_df['USstate'][index] = code
/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/3613544781.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_df['USstate'][index] = code


,USstate,month,YEAR,retweet_count,statuses_count,followers_count,favourites_count,friends_count
0,55,1,2013,0,1609,228,847,222
1,25,1,2013,0,33,0,0,0
2,36,1,2013,1,39932,1015,16,1032
3,12,1,2013,0,2743,108,276,98
4,51,1,2013,2,13105,462,340,70
...,...,...,...,...,...,...,...,...
5054,12,7,2015,0,1304,2083,0,2263
5055,53,7,2015,0,14286,255,4243,220
5056,24,7,2015,65,101939,1706,2350,1697
5057,47,7,2015,53,5548,392,1281,1007


In [324]:
# group by month, year, USstate
twitter_agg1 = twitter_df.groupby(['month','YEAR','USstate']).size().reset_index(name='tweets_counts')
twitter_agg2 = twitter_df.groupby(['month','YEAR','USstate']).sum()
twitter_agg = pd.merge(twitter_agg1, twitter_agg2, on=['month','YEAR','USstate'])
twitter_agg = twitter_agg.rename(columns={'USstate': 'FIPS State Code'})
twitter_agg

,month,YEAR,FIPS State Code,tweets_counts,retweet_count,statuses_count,followers_count,favourites_count,friends_count
0,1,2013,1,4,9,43398,2647,4193,2346
1,1,2013,4,2,3,24673,714,459,724
2,1,2013,5,2,0,48629,490,107,534
3,1,2013,6,25,334,238686,11422,39843,10282
4,1,2013,8,3,0,2056,190,242,916
...,...,...,...,...,...,...,...,...,...
1070,12,2015,40,2,3,108411,1693,33833,1184
1071,12,2015,41,1,6242,36147,529,7747,283
1072,12,2015,42,2,1159,9095,994,5039,873
1073,12,2015,48,21,45015,877925,34591,181835,29483


In [10]:
# use proportion of misogyny tweets
all_tweets = pd.read_excel('./data/All_Tweets_State_Month_Year_05082022.xlsx')
misogyny_tweets = pd.read_excel('./data/Misogyny_Tweets_State_Month_Year_05082022.xlsx')
count_tweets = pd.merge(all_tweets, misogyny_tweets, on = ['USstate', 'year', 'month.keyword: Descending'])
count_tweets['proportion_misogyny_tweets'] = count_tweets['nTweets_y']/count_tweets['nTweets_x']
count_tweets

,USstate,year,month.keyword: Descending,nTweets_x,nTweets_y,proportion_misogyny_tweets
0,ak,2012,10,7795,1,0.000128
1,ak,2012,8,7373,1,0.000136
2,ak,2013,12,6543,1,0.000153
3,ak,2013,9,6300,1,0.000159
4,ak,2013,6,5975,1,0.000167
...,...,...,...,...,...,...
2463,va,2021,8,31318,1,0.000032
2464,va,2021,2,33014,1,0.000030
2465,vt,2012,11,3745,1,0.000267
2466,vt,2012,2,1595,1,0.000627


# 4. Merge All Data

In [342]:
# alcohol and twitter
al_twitter = pd.merge(twitter_agg, alcohol_df, on=['FIPS State Code','YEAR'], how='left')

# add crime
merged_df = pd.merge(al_twitter, crime_agg, on=['FIPS State Code', 'YEAR', 'month'], how='left')
merged_df = merged_df.fillna(0)
merged_df

,month,YEAR,FIPS State Code,tweets_counts,retweet_count,statuses_count,followers_count,favourites_count,friends_count,alcohol_per_month,crime_counts
0,1,2013,1,4,9,43398,2647,4193,2346,80.84,0.0
1,1,2013,4,2,3,24673,714,459,724,25.82,2364.0
2,1,2013,5,2,0,48629,490,107,534,87.82,1763.0
3,1,2013,6,25,334,238686,11422,39843,10282,664.95,28665.0
4,1,2013,8,3,0,2056,190,242,916,568.45,7859.0
...,...,...,...,...,...,...,...,...,...,...,...
1070,12,2015,40,2,3,108411,1693,33833,1184,160.09,5254.0
1071,12,2015,41,1,6242,36147,529,7747,283,64.18,2626.0
1072,12,2015,42,2,1159,9095,994,5039,873,1013.69,32400.0
1073,12,2015,48,21,45015,877925,34591,181835,29483,966.87,145410.0


In [343]:
state_code = pd.read_csv('./data/us-state-ansi-fips.csv')
state_code = state_code[['stname', ' st']]
merged_df = pd.merge(merged_df, state_code, left_on='FIPS State Code', right_on=' st', how='left')
merged_df = merged_df.drop(columns=[' st'])
merged_df = merged_df[merged_df['crime_counts']!=0.0]
merged_df = merged_df.rename(columns={'alcohol_per_month': 'alcohol_counts'})
merged_df

,month,YEAR,FIPS State Code,tweets_counts,retweet_count,statuses_count,followers_count,favourites_count,friends_count,alcohol_counts,crime_counts,stname
1,1,2013,4,2,3,24673,714,459,724,25.82,2364.0,Arizona
2,1,2013,5,2,0,48629,490,107,534,87.82,1763.0,Arkansas
3,1,2013,6,25,334,238686,11422,39843,10282,664.95,28665.0,California
4,1,2013,8,3,0,2056,190,242,916,568.45,7859.0,Colorado
5,1,2013,9,2,4,53082,871,16710,532,131.66,1064.0,Connecticut
...,...,...,...,...,...,...,...,...,...,...,...,...
1070,12,2015,40,2,3,108411,1693,33833,1184,160.09,5254.0,Oklahoma
1071,12,2015,41,1,6242,36147,529,7747,283,64.18,2626.0,Oregon
1072,12,2015,42,2,1159,9095,994,5039,873,1013.69,32400.0,Pennsylvania
1073,12,2015,48,21,45015,877925,34591,181835,29483,966.87,145410.0,Texas


In [344]:
merged_df.to_csv('./data/merged_data.csv')